# 用TensorFlow實作圖片分類
卷積神經網絡（Convolutional Neural Network, CNN）是一種前饋神經網絡，它的人工神經元可以響應一部分覆蓋範圍內的周圍單元，對於大型圖像處理有出色表現。
<img src="img/network.png" width="600px" />
卷積神經網絡由一個或多個卷積層(convolutions layer)和頂端的全連通層（對應經典的神經網絡）組成，同時也包括關聯權重和池化層（pooling layer）。這一結構使得卷積神經網絡能夠利用輸入數據的二維結構。與其他深度學習結構相比，卷積神經網絡在圖像和語音識別方面能夠給出更好的結果。這一模型也可以使用反向傳播算法進行訓練。<br><br>
## 卷積層(convolutions layer)
卷積層就像一台掃模器，一次掃較小的圖塊，掃出來的結果就會是更高維度、大小更小的照片集，如下圖所示。
<img src="img/CNN架構.jpg" width="600px" /><br>
<img src="img/convolutions_2.jpg" width="400px" /><br>
<img src="img/convolutions.jpg" width="400px" /><br>
## 池化層(pooling layer)
池化層是卷積神經網絡中另一個重要的概念，它實際上是一種形式的降採樣。有多種不同形式的非線性池化函數，而其中「最大池化（Max pooling）」是最為常見的。它是將輸入的圖像劃分為若干個矩形區域，對每個子區域輸出最大值。直覺上，這種機制能夠有效地原因在於，在發現一個特徵之後，它的精確位置遠不及它和其他特徵的相對位置的關係重要。池化層會不斷地減小數據的空間大小，因此參數的數量和計算量也會下降，這在一定程度上也控制了過擬合。通常來說，CNN的卷積層之間都會周期性地插入池化層。<br>

池化層通常會分別作用於每個輸入的特徵並減小其大小。目前最常用形式的池化層是每隔2個元素從圖像劃分出2*2的區塊，然後對每個區塊中的4個數取最大值。這將會減少75%的數據量。<br>
<img src="img/pool.jpg" width="400px" />
<img src="img/pool_2.jpg" width="400px" /><br>
經過數個卷積層和池化層後，剩下的就是一塊一塊小圖片，並且厚度會逐漸加厚(圖片數量變多)，那麼這些小圖片就是CNN network抽取出來的特徵，將這些特徵再往下接到MLP network(hidden layer)，最後分成數個類別，再輸出層(output layer)，假設我要分成10個種類，那最後10個layer分別會帶有不同的數字(這些數字加總為1)，代表模組經過學習後，認為這張圖片為哪項分類的機率，而我們當然是選擇機率最大的為最後分類結果(softmax)。

In [1]:
import numpy as np
import pandas as pd
from keras.utils import np_utils
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2

Using TensorFlow backend.


In [2]:
#定義畫圖funtion
import matplotlib.pyplot as plt
def plot_images_labels_predict(images, labels, prediction, idx, num=25):  
    fig = plt.gcf()  
    fig.set_size_inches(12, 14)
    if num > 25: num = 25  
    for i in range(0, num):  
        ax=plt.subplot(5,5, 1+i)  
        ax.imshow(images[idx], cmap='binary')  
        title = "lable=" + str(labels[idx])  
        if len(prediction) > 0:  
            title = "lable={},prediction={}".format(str(labels[idx]), str(prediction[idx]))  
        else:  
            title = "lable={}".format(str(labels[idx])) 
        ax.set_title(title, fontsize=10)  
        ax.set_xticks([]); ax.set_yticks([])
        idx+=1
    plt.show()

In [3]:
#讀取mapping並寫入data frame裡，方便後續做分類、抓取資料
mapping_df = pd.read_csv('//data/examples/may_the_4_be_with_u/where_am_i/mid_term_mapping.txt' ,header=None)
mapping_df.columns = ['folder_name', 'label']
mapping_df.sample(5)

,folder_name,label
13,street,1
4,forest,4
14,tallbuilding,13
1,PARoffice,7
6,industrial,2


In [4]:
test_submit_df = pd.read_csv('//data/examples/may_the_4_be_with_u/where_am_i/img-submission.csv')
#test_submit_df.columns = ['file_name', 'label']
print(len(test_submit_df))

1500


In [5]:
#所有的照片已經預先分類在15個資料夾中，所以透過mapping_df[folder_name]，我們可以造訪所有的train data並記下每張圖片的路徑
import os
image_mapping_path_df = pd.DataFrame(columns=['folder_name', 'label', 'path'])
path = "//data/examples/may_the_4_be_with_u/where_am_i/"
pathData = []
for x in range(0, len(mapping_df['folder_name'])):
    folder_name = mapping_df['folder_name'][x]
    label = mapping_df['label'][x]
    class_folder = path + "train/" + folder_name
    for train_imgName in os.listdir(class_folder):
        train_data_path = class_folder + "/" + train_imgName
        s = pd.DataFrame([[folder_name, label, train_data_path]],columns=['folder_name', 'label', 'path'])
        image_mapping_path_df = image_mapping_path_df.append(s, ignore_index=True)
image_mapping_path_df.sample(5)

,folder_name,label,path
2603,street,1.0,//data/examples/may_the_4_be_with_u/where_am_i...
2169,opencountry,6.0,//data/examples/may_the_4_be_with_u/where_am_i...
77,CALsuburb,9.0,//data/examples/may_the_4_be_with_u/where_am_i...
1101,industrial,2.0,//data/examples/may_the_4_be_with_u/where_am_i...
2219,opencountry,6.0,//data/examples/may_the_4_be_with_u/where_am_i...


In [6]:
#接著利用剛剛記下的路徑，抓取每張圖片，並設定大小
image_width = 256
image_high = 256
#image_cnn_shape是我在cnn層最後每張圖片的大小，因為總共做了4次 pool每次都取2*2，所以長、寬分別剩下1/16
#image_cnn_shape = (image_width//16) * (image_high//16)
image = cv2.imread(image_mapping_path_df["path"][0])
image = cv2.resize(image, (image_width, image_high))
#image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#train = np.reshape(image, (32, 32))
train = np.expand_dims(image, axis=0)
#train = np.reshape(image, (1, 262, 200))
train_label = np.zeros(0, dtype=float)
train_label = np.append(train_label, image_mapping_path_df["label"][0])
for x in tqdm(range(1, len(image_mapping_path_df))):
    path = image_mapping_path_df["path"][x]
    label = image_mapping_path_df["label"][x]
    image = cv2.imread(path)
    image = cv2.resize(image, (image_width, image_high))
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #train = np.reshape(image, (32, 32))
    image = np.expand_dims(image, axis=0)
    train = np.concatenate((train,image), axis=0)
    train_label = np.append(train_label, label)

print("\t[Info] Shape of train data=%s" % (str(train.shape)))
print("\t[Info] Shape of train label=%s" % (str(train_label.shape)))

100%|██████████| 2984/2984 [04:57<00:00, 10.01it/s]

	[Info] Shape of train data=(2985, 256, 256, 3)
	[Info] Shape of train label=(2985,)


In [7]:
y_TrainOneHot = np_utils.to_categorical(train_label) # 將 training 的 label 進行 one-hot encoding
print("y_TrainOneHot.shape = ",y_TrainOneHot.shape)
print(train_label[0]) # 檢視 training labels 第一個 label 的值
y_TrainOneHot[:1] # 檢視第一個 label 在 one-hot encoding 後的結果, 會在第10個位置上為 1, 其他位置上為 0

y_TrainOneHot.shape =  (2985, 15)
9.0


array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.]])

# 切割資料集
把原本的train data切割成train 跟 valid兩塊<br>
train data用來訓練模組 而 valid data用來檢視模組學習成效

In [ ]:
#----------------------------資料training set, testing set 分割---------------------------------------
x_train, x_valid, y_train, y_valid = train_test_split(train,
                                                    y_TrainOneHot,
                                                    test_size = 0.2,
                                                    stratify  = y_TrainOneHot.argmax(axis = 1))
print("training set: %i" % len(x_train))
print("test set: %i" % len(x_valid))

training set: 2388
test set: 597


# 數據增強(ImageDataGenerator)
在train data數量不足時，可以透過數據增強的方式，生成新的數據。<br>
數據增強有很多種方式，例如:選轉圖片、翻轉圖片、將圖片比例縮放、白化、平行拉長...<br>
這邊我實作時，用的是keras的API，在下面附上所有參數。<br>
featurewise_center：布尔值，使输入数据集去中心化（均值为0）, 按feature执行

samplewise_center：布尔值，使输入数据的每个样本均值为0

featurewise_std_normalization：布尔值，将输入除以数据集的标准差以完成标准化, 按feature执行

samplewise_std_normalization：布尔值，将输入的每个样本除以其自身的标准差

zca_whitening：布尔值，对输入数据施加ZCA白化

zca_epsilon: ZCA使用的eposilon，默认1e-6

rotation_range：整数，数据提升时图片随机转动的角度

width_shift_range：浮点数，图片宽度的某个比例，数据提升时图片水平偏移的幅度

height_shift_range：浮点数，图片高度的某个比例，数据提升时图片竖直偏移的幅度

shear_range：浮点数，剪切强度（逆时针方向的剪切变换角度）

zoom_range：浮点数或形如[lower,upper]的列表，随机缩放的幅度，若为浮点数，则相当于[lower,upper] = [1 - zoom_range, 1+zoom_range]

channel_shift_range：浮点数，随机通道偏移的幅度

fill_mode：；‘constant’，‘nearest’，‘reflect’或‘wrap’之一，当进行变换时超出边界的点将根据本参数给定的方法进行处理

cval：浮点数或整数，当fill_mode=constant时，指定要向超出边界的点填充的值

horizontal_flip：布尔值，进行随机水平翻转

vertical_flip：布尔值，进行随机竖直翻转

rescale: 重放缩因子,默认为None. 如果为None或0则不进行放缩,否则会将该数值乘到数据上(在应用其他变换之前)

preprocessing_function: 将被应用于每个输入的函数。该函数将在图片缩放和数据提升之后运行。该函数接受一个参数，为一张图片（秩为3的numpy array），并且输出一个具有相同shape的numpy array

data_format：字符串，“channel_first”或“channel_last”之一，代表图像的通道维的位置。该参数是Keras 1.x中的image_dim_ordering，“channel_last”对应原本的“tf”，“channel_first”对应原本的“th”。以128x128的RGB图像为例，“channel_first”应将数据组织为（3,128,128），而“channel_last”应将数据组织为（128,128,3）。该参数的默认值是~/.keras/keras.json中设置的值，若从未设置过，则为“channel_last”
<br>
參考資料:https://keras-cn.readthedocs.io/en/latest/preprocessing/image/<br>
        https://zhuanlan.zhihu.com/p/30197320

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center = False,
    samplewise_center = False,
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    rotation_range = 30,
    shear_range = 0.4,
    horizontal_flip = True,
    vertical_flip = False,
    zca_whitening = False,
    channel_shift_range = 0.0)

# 計算特徵正規化所需的數量
datagen.fit(x_train)
x_IDG_train = x_train
y_IDG_label = y_train

#決定我要生成幾倍的資料量
for e in range(4):
    print('Data Gene epochs =',e)
    batches=0
    per_batch=32
    #每次的batch_size生成數據
    for x_batch,y_batch in datagen.flow(x_train,y_train,batch_size=32):
        x_IDG_train = np.concatenate((x_IDG_train, x_batch), axis = 0)
        y_IDG_label = np.concatenate((y_IDG_label, y_batch), axis = 0)
        batches += 1
        if batches >= len(train) / per_batch:
        # 需要手動跳出迴圈
            break
print('Data Augment(x_IDG_train) total shape =',x_IDG_train.shape)
print('Data Augment(y_IDG_label) total shape =',y_IDG_label.shape)
print("-------------------")

Data Gene epochs = 0
Data Gene epochs = 1


In [ ]:
plot_images_labels_predict(x_IDG_train,y_IDG_label, [], 0)

# 正規化
正規化對CNN模組在學習上有"非常非常非常"大的重要性，如果在沒有做正規化的情況下training model，那麼model學習的效率會非常低，幾乎學不起來。選擇除以255的原因在於，圖片上每個像素的值屆於0~255之間，除以255可以讓所有值屆於0~1之間。

In [ ]:
# Normalization
#x_train = x_train/255
x_IDG_train = x_IDG_train /255
x_valid = x_valid/255

#print("\t[Info] x_train: %s" % (str(x_train.shape)))
print("\t[Info] x_IDG_train: %s" % (str(x_IDG_train.shape)))
print("\t[Info] y_IDG_label: %s" % (str(y_IDG_label.shape)))
#x_train[0]
x_IDG_train[0]

In [ ]:
plot_images_labels_predict(x_IDG_train,y_IDG_label, [], 0)

# 定義模組
以下例子中，我用tensorflow架構了四層convolutions layer，並在每一層後面，都接上pool 2*2，做完pool要進入下一層之前，drop 25%神經元，讓模組在學習時，不會太容易overfitting。在CNN之後，我接上兩層的MLP，第一層 layer數量是512，第二層 layer數量是128，最後經過softmax分類成15類。(out put layer)<br>
## drop
drop在解決overfittimg時，是非常有用的方式，因為model在每次進入下一層之前，都遺失了一些訊號，那麼自然不可能過度學習啦!

In [ ]:
#tf.reset_default_graph()
#sess = tf.InteractiveSession()

In [ ]:
from __future__ import print_function
import tensorflow as tf

batch_size = 32
epochs = 50
lr = 0.0001

hidden1_neurons = 512
hidden2_neurons = 256
image_cnn_shape = (image_width//32) * (image_high//32)


with tf.name_scope('input'):
    xs = tf.placeholder(shape = [None, image_width, image_high, 3],
                        dtype = tf.float32,
                        name = 'xs')
    ys = tf.placeholder(shape = [None, 15],
                        dtype = tf.float32,
                        name = 'ys')
    keep_prob = tf.placeholder(dtype = tf.float32,
                              name = 'keep_prob')
## conv1 layer ##
with tf.variable_scope('conv1'):
    conv1_1 = tf.layers.conv2d(inputs=xs, filters=32, strides=(1, 1), kernel_size=[3,3], padding="same",
                               activation=tf.nn.relu)
    conv1_2 = tf.layers.conv2d(inputs=conv1_1, filters=32, strides=(1, 1), kernel_size=[3,3], padding="same",
                               activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1_2, strides=2, pool_size=[2,2])

## conv2 layer ##
with tf.variable_scope('conv2'):
    conv2_1 = tf.layers.conv2d(inputs=pool1, filters=64, strides=(1, 1), kernel_size=[3,3], padding="same",
                               activation=tf.nn.relu)
    conv2_2 = tf.layers.conv2d(inputs=conv2_1, filters=64, strides=(1, 1), kernel_size=[3,3], padding="same",
                               activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2_2, strides=2, pool_size=[2,2])

# conv3 layer ##
with tf.variable_scope('conv3'):
    conv3_1 = tf.layers.conv2d(inputs=pool2, filters=128, strides=(1, 1), kernel_size=[3,3], padding="same",
                               activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3_1, strides=2, pool_size=[2,2])

# conv4 layer ##
with tf.variable_scope('conv4'):
    conv4_1 = tf.layers.conv2d(inputs=pool3, filters=256, strides=(1, 1), kernel_size=[3,3], padding="same",
                               activation=tf.nn.relu)
    pool4 = tf.layers.max_pooling2d(inputs=conv4_1, strides=2, pool_size=[2,2])

# conv5 layer ##
with tf.variable_scope('conv5'):
    conv5_1 = tf.layers.conv2d(inputs=pool4, filters=512, strides=(1, 1), kernel_size=[3,3], padding="same",
                               activation=tf.nn.relu)
    pool5 = tf.layers.max_pooling2d(inputs=conv5_1, strides=2, pool_size=[2,2])

with tf.variable_scope('hidden_layer'):
    pool5_flat = tf.reshape(pool5, [-1, image_cnn_shape*512])
    hidden_1 = tf.layers.dense(inputs=pool5_flat, units=512, activation=tf.nn.relu)
    hidden_2 = tf.layers.dense(inputs=hidden_1, units=256, activation=tf.nn.relu)
    
with tf.variable_scope('output_layer'):
    output = tf.layers.dense(inputs=hidden_1, units=15, activation=tf.nn.relu)
    pre = tf.nn.softmax(output)

with tf.name_scope('cross_entropy'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=ys))

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(output),1), tf.argmax(ys,1)) #如果答案對則回傳true
    compute_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #將回傳的true/false轉乘1/0並計算平均(計算正確率)

with tf.name_scope('train'):
    #使用adam做optimization最小化loss funciotn(不斷取微分並逼近local min)
    train_step = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

In [ ]:
train_loss_list, valid_loss_list = [], []
train_acc_list, valid_acc_list = [], []

sess = tf.Session()
#tf.reset_default_graph()
#sess = tf.InteractiveSession()
# important step
# tf.initialize_all_variables() no long valid from
init = tf.global_variables_initializer()
sess.run(init)

for i in tqdm(range(epochs)):
    # get batch
    total_batch = int(np.floor(len(x_IDG_train) / batch_size)) # just drop out last few samples...
    
    train_loss_collector, train_acc_collector = [], []
    for j in np.arange(total_batch):
        batch_idx_start = j * batch_size
        batch_idx_stop = (j+1) * batch_size

        x_batch = x_IDG_train[batch_idx_start : batch_idx_stop]
        y_batch = y_IDG_label[batch_idx_start : batch_idx_stop]
        
        this_loss, this_acc, _ = sess.run([loss, compute_acc, train_step], feed_dict = {xs: x_batch, ys: y_batch, keep_prob: 1})
        train_loss_collector.append(this_loss)
        train_acc_collector.append(this_acc)
            
    # do validation at the end of each epoch
    valid_total_batch = int(np.floor(len(x_valid) / batch_size))
    valid_loss_collector, valid_acc_collector = [], []
    for j in np.arange(valid_total_batch):
        batch_idx_start = j * batch_size
        batch_idx_stop = (j+1) * batch_size

        x_batch = x_valid[batch_idx_start : batch_idx_stop]
        y_batch = y_valid[batch_idx_start : batch_idx_stop]
        
        this_acc, this_loss = sess.run([compute_acc, loss], feed_dict = {xs: x_batch, ys : y_batch, keep_prob: 1})
        valid_acc_collector.append(this_acc)
        valid_loss_collector.append(this_loss)
    
    valid_loss_list.append(np.mean(valid_loss_collector))
    valid_acc_list.append(np.mean(valid_acc_collector))
    train_loss_list.append(np.mean(train_loss_collector))
    train_acc_list.append(np.mean(train_acc_collector))

    # at the end of each epoch, shuffle the data
    x_IDG_train, y_IDG_label = shuffle(x_IDG_train, y_IDG_label)

# At the end of the training, do testing set
#result = sess.run(pre, feed_dict={xs: x_valid, ys: y_valid, keep_prob: 1})
print('--- training done ---')

saver = tf.train.Saver()
model_path = "model_cnn/model.ckpt"
save_path = saver.save(sess, model_path)

print("--- save done ---")

In [ ]:
print('accuracy: %.2f' % valid_acc_list[len(valid_acc_list)-1])
#--------------------------------------plot---------------------------------------------
print("loss")
plt.plot(np.arange(len(train_loss_list)), train_loss_list, 'b', label = 'train')
plt.plot(np.arange(len(valid_loss_list)), valid_loss_list, 'r', label = 'valid')
plt.legend()
plt.show()

print("accuracy")
plt.plot(np.arange(len(train_acc_list)), train_acc_list, 'b', label = 'train')
plt.plot(np.arange(len(valid_acc_list)), valid_acc_list, 'r', label = 'valid')
plt.legend(loc = 4)
plt.show()

# 混淆矩陣
我們可以透過混淆矩陣，觀察分類錯誤的情況。

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
#Y_pred = model.predict(x_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(result, axis = 1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_valid, axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(15))

<img src="img/result.jpg"/>

# 後言：
在建立model的過程中，一開始用兩層的convolution作為model，這時觀察會發現，圖片像素越少(圖片越小)，model準確率變高，然而這樣辨識率的極限大約(25%左右)，再想要獲得更好的辨識率，勢必需要調高圖片的像素。
## 關於圖片大小選用
圖片越大，model能學習的資源越多，因此為了提取更精準的特徵，勢必也要加深CNN捲積的部分。
## train loss Nan
在加深捲積層時，發現model有學不起來來的現象，在第1個epochs時，train loss極劇下降到接近0，之後就不會在變動了。這時觀察混淆矩陣會發現模組將大部分的資料，都分類到同個分類，也就是說 模組過於執著。上網查資料後發現，最常見的原因是學習率太高，導致模組「顽固」認為數據屬於錯誤的類別，而正確的類別機率為0(小數點下溢出)，這樣用交叉熵就會算出無窮大的損失函數。一旦出現這種情況，無窮大對參數求值就會變成NaN，之後整個網路的參數都變成NaN了。<b>為了解決這個問題，調降learn rate有效降低這個現象。</b>
<img src="img/lossNan.png"/>
參考資料：https://www.zhihu.com/question/62441748
## 捲積層大小
這會影響一個捲積層視野的大小，為了讓filter看到更大片的圖形，一開始我採用5*5的filter，發現model學習的結果不好，參考VGG-16、VGG-19的論文結論，改成採用兩層3*3的filter。<br>
原因有以下幾點:<br>
1：3x3是最小的能夠捕獲上下左右的最小單位。 <br>
2：兩個3x3的捲積層的視野是5x5；三個3x3的視野是7x7，可以替代大的filter尺寸。 <br>
<img src="img/filter.png" width="400px"/>
<center>最左上角的像素對3x3、5x5的視野範圍</center>
<img src="img/filter2.png" width="400px"/>
<center>紅色的區塊是3x3兩層的範圍</center><br>
可以看出對左上角(黃色)的像素來說，3x3兩層與5x5的範圍一致<br>
3：多個3x3的捲積層比一个大尺寸filter捲積層有更多的非線性(激發函數)，使得判决函数更加具有判决性。<br>
4：多個3x3的捲積層比一个大尺寸的filter有更少的参数，假设捲積層的输入和输出的特征圖大小相同為C，那麼三個3x3的捲積層参數個數為:3x（3x3xCxC）=27CC；一個7x7的捲積層參數為49CC；所以可以把三個3x3的filter看成是一個7x7filter的分解，又有更多的非線性(激發函數)。

參考資料：https://blog.csdn.net/u011440696/article/details/77756776